# 6. Advanced RAG


In [1]:
!pip install numpy==1.26.4

In [5]:
# 【注意】
# 上記の `!pip install numpy==1.26.4` を実行したあと、
# Google Colab 上部のメニューから「ランタイム」の「セッションを再起動する」を実行してください。
# その後このセルを実行して `1.26.4` と表示されることを確認してください。

import numpy as np

print(np.__version__)
assert np.__version__ == "1.26.4"

1.26.4


In [10]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

## 6.2. ハンズオンの準備


In [4]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 \
    langchain-community==0.3.0 GitPython==3.1.43 \
    langchain-chroma==0.1.4 tavily-python==0.5.0 pydantic==2.10.6
# 📦 すべてのLangChain関連ライブラリをまとめて最新の安定版にアップデートするよ！
!pip install -U langchain langchain-core langchain-openai langchain-community langsmith

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-text-splitters to deter

In [6]:
from langchain_community.document_loaders import GitLoader  # GitHubからファイルを取ってくる道具を使うよ

def file_filter(file_path: str) -> bool:  # 「このファイルを使う？」って聞かれたときのルールを作るよ
    return file_path.endswith(".mdx")  # 最後が「.mdx」のファイルだけ「使うよ！」って返すよ

loader = GitLoader(  # GitHubからファイルを読み込む準備をするよ
    clone_url="https://github.com/langchain-ai/langchain",  # GitHubのどの場所から取るかを指定するよ
    repo_path="./langchain",  # ファイルをダウンロードして保存する場所を決めるよ
    branch="master",  # 「master」という最新のバージョンから取ってくるよ
    file_filter=file_filter,  # さっき作った「.mdxだけ使うよ」というルールを使うよ
)

documents = loader.load()  # GitHubからファイルを取り出して読み込むよ（.mdxだけ！）
print(len(documents))  # 読み込んだファイルの数を教えてくれるよ


418


In [7]:
from langchain_chroma import Chroma  # Chroma（クローマ）っていう引き出し型データベースを使うよ
from langchain_openai import OpenAIEmbeddings  # 文章を特徴で表す道具（埋め込み）を使うよ

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 「文章の特徴」を作るための道具を用意するよ（小さいモデルを使うよ）

db = Chroma.from_documents(documents, embeddings)  # 読み込んだドキュメントを特徴にして、Chromaに入れるよ（あとで検索できるようになるよ）


ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
ERROR:chromadb.telemetry.product.posthog:Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


In [13]:
from langchain_core.output_parsers import StrOutputParser  # AIの答えを文字にしてくれるよ
from langchain_core.prompts import ChatPromptTemplate  # AIに渡す質問のテンプレートを作る道具だよ
from langchain_core.runnables import RunnablePassthrough  # 入力をそのまま渡すための道具だよ（何も加工しない）
from langchain_openai import ChatOpenAI  # OpenAIのAI（gpt-4o-mini）を使うよ

# 💬 AIに渡す質問テンプレートを作るよ
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

# 🧠 AIのモデル（gpt-4o-mini）を準備するよ
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # temperature=0 でブレずに答えてくれるよ

# 📚 ベクトルデータベースから、関連する情報を探すretrieverを作るよ
retriever = db.as_retriever()  # さっきChromaに入れたドキュメントを使うよ

# 🔗 処理の流れ（チェーン）を作るよ
chain = {
    "question": RunnablePassthrough(),  # ユーザーの質問をそのまま使うよ
    "context": retriever,  # 質問に合う文脈（情報）を探すよ
} | prompt | model | StrOutputParser()  # テンプレートに入れて → AIに渡して → 答えを文字にしてくれるよ

# 🚀 実際に質問してみよう！
output=chain.invoke("LangChainの概要を教えて")  # 「LangChainってなに？」と聞いて、文脈をもとにAIが答えてくれるよ
print(output)  # 答えを表示するよ

LangChainは、大規模言語モデル（LLM）を活用したアプリケーションを開発するためのフレームワークです。このフレームワークは、LLMアプリケーションのライフサイクルの各段階を簡素化します。具体的には、以下のような機能を提供しています。

1. **開発**: LangChainのオープンソースコンポーネントやサードパーティ統合を使用してアプリケーションを構築できます。LangGraphを利用することで、状態を持つエージェントを作成し、ストリーミングや人間の介入をサポートします。

2. **生産化**: LangSmithを使用してアプリケーションを検査、監視、評価し、継続的に最適化して自信を持ってデプロイできます。

3. **デプロイ**: LangGraphアプリケーションを生産準備が整ったAPIやアシスタントに変換できます。

LangChainは、LLMや関連技術（埋め込みモデルやベクトルストアなど）に対する標準インターフェースを実装し、数百のプロバイダーと統合しています。また、複数のオープンソースライブラリで構成されており、ユーザーは自分のニーズに応じてコンポーネントを選択して使用できます。


## 6.3. 検索クエリの工夫


### HyDE（Hypothetical Document Embeddings）


In [ ]:
hypothetical_prompt = ChatPromptTemplate.from_template("""\  # 質問のテンプレートを作るよ（「この質問に答えてね！」っていう形を決める）
次の質問に回答する一文を書いてください。  # AIに「1文だけで答えてね」と伝えてるよ

質問: {question}  # 実際の質問の場所を {question} っていう変数にしてるよ
""")

hypothetical_chain = hypothetical_prompt | model | StrOutputParser()  # 作ったテンプレートにAIと答えを整える道具をつなげて、1つの流れ（チェーン）を作るよ


In [ ]:
hyde_rag_chain = {  # HyDE（まず考えてから探す）型のRAGチェーンを作るよ
    "question": RunnablePassthrough(),  # ユーザーの質問をそのまま「question」に入れるよ
    "context": hypothetical_chain | retriever,  # まずAIが一文で予想→その文章を使って検索（retriever）するよ
} | prompt | model | StrOutputParser()  # 質問と検索結果を使って、AIに答えを考えてもらって、読みやすい形に整えるよ

hyde_rag_chain.invoke("LangChainの概要を教えて")  # 「LangChainってなに？」って質問して、AIに答えてもらうよ


### 複数の検索クエリの生成


In [ ]:
from pydantic import BaseModel, Field  # データの型をきちんと決める道具を使うよ（間違えないようにするため）

# 🔧 出力の形を決めるよ（ここでは検索クエリを3つにする）
class QueryGenerationOutput(BaseModel):  # 出てくる結果のルールを決めるよ
    queries: list[str] = Field(..., description="検索クエリのリスト")  # クエリ（質問文）を3つもらうよ

# 🧠 AIにお願いするメッセージを作るよ（質問に対して3つの違うクエリを考えてって伝える）
query_generation_prompt = ChatPromptTemplate.from_template("""\
質問に対してベクターデータベースから関連文書を検索するために、
3つの異なる検索クエリを生成してください。
距離ベースの類似性検索の限界を克服するために、
ユーザーの質問に対して複数の視点を提供することが目標です。

質問: {question}
""")

# 🧩 メッセージ → AIモデル（答えを作る）→ 決まった形で取り出す → リストにする
query_generation_chain = (  # チェーンを作るよ（つなげて一気に処理する）
    query_generation_prompt  # AIへのお願い文を使うよ
    | model.with_structured_output(QueryGenerationOutput)  # 結果を「決まった形」で受け取るよ（QueryGenerationOutputの形に）
    | (lambda x: x.queries)  # その中から「クエリのリスト」だけを取り出すよ
)


In [ ]:
multi_query_rag_chain = {  # 「複数のクエリで検索して答える」ためのチェーンを作るよ
    "question": RunnablePassthrough(),  # ユーザーの質問をそのまま通すよ（フィルターしない）
    "context": query_generation_chain | retriever.map(),  # いろんな聞き方（クエリ）を作って、それぞれ検索するよ
} | prompt | model | StrOutputParser()  # 文脈を入れてAIに聞いて、きれいな答えを取り出すよ

multi_query_rag_chain.invoke("LangChainの概要を教えて")  # 「LangChainって何？」って質問して、AIに答えてもらうよ


## 6.4. 検索後の工夫


### RAG Fusion


In [ ]:
from langchain_core.documents import Document  # ドキュメント（文章のまとまり）を扱うための型を読み込むよ

def reciprocal_rank_fusion(
    retriever_outputs: list[list[Document]],  # 検索結果（複数クエリ×それぞれのドキュメント）をリストで受け取るよ
    k: int = 60,  # スコアの調整用の数字（大きいほど差が小さくなる）を設定するよ
) -> list[str]:  # 最終的に選ばれたドキュメントの本文（文字列）をリストで返すよ
    content_score_mapping = {}  # 文章とスコアをセットで記録するノートを作るよ

    for docs in retriever_outputs:  # 検索クエリごとの結果を1つずつ見ていくよ
        for rank, doc in enumerate(docs):  # それぞれのドキュメントに順位をつけながらループするよ
            content = doc.page_content  # ドキュメントの中身（文章）を取り出すよ

            if content not in content_score_mapping:  # はじめて見る文章だったら…
                content_score_mapping[content] = 0  # スコアを0でスタートさせるよ

            content_score_mapping[content] += 1 / (rank + k)  # 順位に応じたスコアを加算するよ（順位が上ほどスコアが高い）

    ranked = sorted(content_score_mapping.items(), key=lambda x: x[1], reverse=True)  # スコアが高い順に並べ替えるよ
    return [content for content, _ in ranked]  # 並べ替えた結果から、文章の部分だけを取り出して返すよ


In [ ]:
rag_fusion_chain = {  # RAG Fusion（質問→検索→まとめる）をするチェーンを作るよ
    "question": RunnablePassthrough(),  # 「質問」をそのまま次の処理に渡すだけの道（パススルー）だよ
    "context": query_generation_chain | retriever.map() | reciprocal_rank_fusion,  # クエリを3つ作って検索 → 結果を融合するよ
} | prompt | model | StrOutputParser()  # プロンプト → モデル → 出力の整形、までつなげるよ（パイプで順番に渡す）

rag_fusion_chain.invoke("LangChainの概要を教えて")  # 「LangChainの概要を教えて」って質問を実行してみるよ！


### Cohere のリランクモデルを使用する準備


In [ ]:
os.environ["COHERE_API_KEY"] = userdata.get("COHERE_API_KEY")

In [ ]:
!pip install langchain-cohere==0.3.0  # LangChainでCohereというAIを使うための道具をバージョン0.3.0で入れるよ


### Cohere のリランクモデルの導入


In [ ]:
from typing import Any  # いろんな型（データの種類）を使うための道具だよ

from langchain_cohere import CohereRerank  # CohereっていうAIの「再ランク付け」ツールを使うよ
from langchain_core.documents import Document  # 文書（ドキュメント）を扱うための型を読み込むよ


# 🧠 文章を重要な順に並び替える関数を作るよ（上位top_n個だけを使うよ）
def rerank(inp: dict[str, Any], top_n: int = 3) -> list[Document]:  # 入力は辞書（質問と文書が入ってる）だよ
    question = inp["question"]  # 質問の内容を取り出すよ
    documents = inp["documents"]  # 文書のリストを取り出すよ

    cohere_reranker = CohereRerank(model="rerank-multilingual-v3.0", top_n=top_n)  # Cohereの「再ランク付け」モデルを使うよ（3つ選ぶ）
    return cohere_reranker.compress_documents(documents=documents, query=question)  # 質問にぴったりの文書だけを残すよ


# 🤖 Cohereで再ランク → AIに聞く、という流れを作るよ
rerank_rag_chain = (
    {
        "question": RunnablePassthrough(),  # 質問をそのまま渡すよ
        "documents": retriever,  # 検索で見つけた文書を入れるよ
    }
    | RunnablePassthrough.assign(context=rerank)  # rerank関数で文書をしぼって、「context」に入れるよ
    | prompt  # AIに渡すメッセージの形を整えるよ
    | model  # AIモデルを使って答えを考えてもらうよ
    | StrOutputParser()  # 答えをきれいな文字だけに整えるよ
)

# 🧪 実際に「LangChainの概要を教えて」と聞いてみるよ
rerank_rag_chain.invoke("LangChainの概要を教えて")  # AIからの答えが返ってくるよ！


## 6.5. 複数の Retriever を使う工夫


### LLM によるルーティング


In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever  # Tavilyっていうインターネット検索の道具を読み込むよ

# 🔍 自分で用意した資料から探すretrieverに「名前（run_name）」をつけるよ
langchain_document_retriever = retriever.with_config(
    {"run_name": "langchain_document_retriever"}  # このretrieverの記録名を「langchain_document_retriever」にするよ
)

# 🌐 インターネットから3件まで調べてくれるretrieverを作るよ
web_retriever = TavilySearchAPIRetriever(k=3).with_config(
    {"run_name": "web_retriever"}  # このretrieverの記録名を「web_retriever」にするよ
)


In [ ]:
from enum import Enum  # Enum（えぬむ）という「選べる値のリスト」を使うために読み込むよ


# 🔀 Retrieverの種類を「名前つき」で定義するよ（ふたつの選択肢があるよ）
class Route(str, Enum):
    langchain_document = "langchain_document"  # 自分の資料から探す道
    web = "web"  # インターネットから探す道


# ✅ どの道を選んだかを表す箱（モデル）を作るよ
class RouteOutput(BaseModel):
    route: Route  # routeっていう項目に、どっちの道を選んだか入るよ


# 🧠 AIに「どっちの道（retriever）を使う？」って聞くための質問テンプレートを作るよ
route_prompt = ChatPromptTemplate.from_template("""\
質問に回答するために適切なRetrieverを選択してください。

質問: {question}
""")

# 🔗 チェーンを作るよ：「質問テンプレート → AIに判断させる → routeの値だけ取り出す」って流れだよ
route_chain = (
    route_prompt  # AIへの質問テンプレートを使うよ
    | model.with_structured_output(RouteOutput)  # 答えは「RouteOutput」の形で出してもらうよ
    | (lambda x: x.route)  # 最後に「どの道か？」だけ取り出すよ
)


In [ ]:
def routed_retriever(inp: dict[str, Any]) -> list[Document]:  # AIが選んだ道に応じて情報を取りに行く関数を作るよ
    question = inp["question"]  # ユーザーが聞いた質問を取り出すよ
    route = inp["route"]  # AIが選んだ道（ルート）を取り出すよ

    if route == Route.langchain_document:  # 「自分の資料から答える道」のとき
        return langchain_document_retriever.invoke(question)  # 資料検索リトリーバーで答えを探すよ
    elif route == Route.web:  # 「インターネットから答える道」のとき
        return web_retriever.invoke(question)  # Web検索リトリーバーで答えを探すよ

    raise ValueError(f"Unknown route: {route}")  # もし知らない道だったらエラーにするよ


# 🛤️ 最後に全部つなげた「道の選択つきRAGチェーン」を作るよ
route_rag_chain = (
    {
        "question": RunnablePassthrough(),  # ユーザーの質問をそのまま渡すよ
        "route": route_chain,  # どの道を使うかAIに判断してもらうよ
    }
    | RunnablePassthrough.assign(context=routed_retriever)  # 選んだ道に応じて答えを取ってくるよ
    | prompt  # AIに「文脈と質問」を渡すテンプレートを使うよ
    | model  # AIモデルに答えてもらうよ
    | StrOutputParser()  # 答えをキレイな文章に変えるよ
)


In [ ]:
# 🧪 質問1：LangChainについて説明してもらうよ
route_rag_chain.invoke("LangChainの概要を教えて")

In [ ]:
# 🧪 質問2：東京の天気について聞いてみるよ
route_rag_chain.invoke("東京の今日の天気は？")

### ハイブリッド検索の実装


In [ ]:
!pip install rank-bm25==0.2.2  # BM25（似た文章を見つける検索方法）を使うための準備だよ



In [ ]:

from langchain_community.retrievers import BM25Retriever  # BM25方式で検索する道具を使うよ

chroma_retriever = retriever.with_config(  # いつものベクトル検索リトリーバー（Chroma）に名前をつけるよ
    {"run_name": "chroma_retriever"}  # 名前は「chroma_retriever」だよ
)

bm25_retriever = BM25Retriever.from_documents(documents).with_config(  # ドキュメントからBM25検索用の道具を作るよ
    {"run_name": "bm25_retriever"}  # 名前は「bm25_retriever」だよ
)


In [ ]:

from langchain_core.runnables import RunnableParallel  # 同時にふたつの処理を動かす仕組みを使うよ

hybrid_retriever = (  # ベクトル検索とBM25検索を合体させた「ハイブリッド検索機能」だよ
    RunnableParallel({  # 2つのリトリーバーを同時に動かすよ
        "chroma_documents": chroma_retriever,  # 1つめはChroma（ベクトル検索）だよ
        "bm25_documents": bm25_retriever,  # 2つめはBM25（似た文章検索）だよ
    })
    | (lambda x: [x["chroma_documents"], x["bm25_documents"]])  # 出てきた検索結果をリストにまとめるよ
    | reciprocal_rank_fusion  # 2つの結果をうまく合体させて、いい順に並べ直すよ
)

In [ ]:

hybrid_rag_chain = (  # ここで「質問して答えるチェーン」を作るよ（ハイブリッド検索付き！）
    {
        "question": RunnablePassthrough(),  # ユーザーの質問をそのまま通すよ
        "context": hybrid_retriever,  # 上で作ったハイブリッド検索の結果を使うよ
    }
    | prompt  # 文脈と質問をセットでAIに渡すテンプレートを使うよ
    | model  # AIモデルに考えてもらうよ
    | StrOutputParser()  # 答えをきれいに文章に整えるよ
)

hybrid_rag_chain.invoke("LangChainの概要を教えて")  # 「LangChainってなに？」って聞いてみるよ